Author: Micheal C. Chen

Contact: muchuchen03@gmail.com

# Task 1
In this task, we will write a program to diagonalize the **Cluster-Ising model** on a 1D 8-site ring.
$$
H=-\sum_{j=1}^{N}\Big(
g\sigma^z_{j-1}\sigma^y_{j}\sigma^z_{j+1}
+J\sigma^z_{j}\sigma^z_{j+1}
+h\sigma^x_{j}
\Big)
$$
<p align="center">
  <img src="ring-chain.png" alt="1D Cluster-Ising Model on a 8-site ring" width="360">
  
</p>

Use the parameters: $g=1, J=1, h=1$

and use **translation symmetry**.

## Tasks

1. **Dispersion relation:** Calculate the dispersion relation $E(k)$ of this model and show the eigenvalues in each momentum sector.
2. **Ground state:** Calculate the ground state energy $E$.
3. **Magnetization:** Using the true ground state, compute $\langle \sigma_i^z\rangle$ and $\langle \sigma_i^x\rangle$.

## Bit Operation
The most significant part of exact diagonalization is defining bit operations.

In [2]:
def ReadBit(i, n): # Read n-th bit
    return (i&(1<<n))>>n

def FlipBit(i, n): # Filp n-th bit 
    return i^(1<<n)

def PickBit(i, k, n): # Pick up nbits from k-th bit
    return (i&((2**n-1)<<k))>>k

def RotLBit(i, L, n): # circular bit shift left (for construcing translation operator)
    return (PickBit(i, 0, L-n)<<n) + (i>>(L-n))

In [3]:
import numpy as np
import numpy.linalg as LA
from scipy import sparse

## Hop List Construction
The Hamiltonian has Cluster interactions, Ising interactions and externakl field term. So we define corresponding HopList.

In [4]:
def IsingHopList(Ns):
    """
    Args:
        Ns: Number of sites

    Returns:
        IsingHopList: List of pairs of sites for Ising interaction (Periodic boundary condition)
    """
    IsingHopList = []
    for site in range(Ns):
        IsingHopList.append([site, (site+1) % Ns]) # Periodic boundary condition
    return IsingHopList

def ClusterHopList(Ns):
    """
    Args:
        Ns: Number of sites
        
    Returns:
        ClusterHopList: List of triplets of sites for cluster interaction (Periodic boundary condition)
    """
    ClusterHopList = []
    for site in range(Ns):
        ClusterHopList.append([site, (site+1)%Ns, (site+2)%Ns]) # Periodic boundary condition
    return ClusterHopList

## Hamiltonian and Translation Matrix Construction
In this part I will construct sparse matrix form of Hamiltonian and Translation matrix with the definition $T|abcd\rangle = |bcda\rangle$ with only bit operations.

The **Hamiltonian** has off-diagonal terms and diagonal terms:
* Off-diagonal term: $-\sum_j \left(g\sigma^z_{j-1}\sigma^y_{j}\sigma^z_{j+1}+h\sigma^x_{j}\right)$
* Diagonal term: $-\sum_j J\sigma^z_{j}\sigma^z_{j+1}$

The **Translation matrix** only has off-diagonal terms

In [5]:
def ClusterIsingHam(Params, Ns):
    """
    Args:
        Params: List of parameters [g, J, h]
        Ns: Number of sites
    Returns:
        Hamr: Sparse Hamiltonian matrix 
    """
    g, J, h = Params
    dim = 2 ** Ns
    IsingList = IsingHopList(Ns)
    ClusterList = ClusterHopList(Ns)
    HamFrom = []
    HamTo = []
    HamValue = []
    
    for basis in range(dim):
        # Off-diagonal Term
        for ih in range(len(ClusterList)):
            Clu0 = ClusterList[ih][0]
            Clu1 = ClusterList[ih][1]
            Clu2 = ClusterList[ih][2]
            newbasis = FlipBit(basis, Clu1)
            HamTo.append(newbasis)
            HamFrom.append(basis)
            Val = -g * (2 * ReadBit(basis, Clu0) -1 ) *1j *  (2 * ReadBit(basis, Clu1)- 1) * (2 * ReadBit(basis, Clu2) - 1) - h
            HamValue.append(Val)
        # Diagonal Term
        for ih in range(len(IsingList)):
            Pos0 = IsingList[ih][0]
            Pos1 = IsingList[ih][1]

            HamTo.append(basis)
            HamFrom.append(basis)
            HamValue.append(-J * (2 * ReadBit(basis, Pos0) -1 ) * (2 * ReadBit(basis, Pos1) - 1))
    
    Hamr = sparse.coo_matrix((HamValue,(HamTo, HamFrom)), shape=(dim, dim)).toarray()

    return Hamr

def TranslationMat(Ns):
    """
    Args:
        Ns: Number of sites
    Returns:
        TransMat: Translation operator matrix
    """
    dim = 2 ** Ns
    TransTo = []
    TransFrom = []
    TransValue = []
    for basis in range(dim):
        newbasis = RotLBit(basis, Ns, 1)
        TransTo.append(newbasis)
        TransFrom.append(basis)
        TransValue.append(1)

    TransMat = sparse.coo_matrix((TransValue,(TransTo, TransFrom)), shape=(dim, dim)).toarray()

    return TransMat

To get $E(k)$ in practice, we notice $[T, H]=0$ thus they share same eigenvectors. Directly diagonalizing $H$ and $T$ **separately** can be numerically awkward in degenerate subspaces (eigenvectors are not uniquely fixed). To obtain **concurrent eigenvectors** stably, diagonalize the linear combination then we decide to solve the eigenvaluye problem:
$$
(H-\alpha e^{-ik} T)|\psi\rangle = \lambda |\psi\rangle
$$

If $|\psi\rangle=|E,k\rangle$ is a simultaneous eigenstate of $H$ and $T$, then eigenvalue is $\big(E-\alpha \big)$
so $|\psi\rangle$ is automatically an eigenvector of $H-\alpha e^{-ik} T$. For **generic** $\alpha$, distinct pairs $(E,k)$ map to **distinct** complex numbers $E-\alpha$, making the spectrum of $H-\alpha e^{-ik} T$ simple (non-degenerate) and thus fixing the eigenvectors uniquely.

After diagonalizing and obtaining the eigenvectors $\{|\phi_j\rangle\}$, recover the physical energy and momentum via **Rayleigh quotients**:
$$
E_j=\langle\phi_j|H|\phi_j\rangle\in\mathbb R,\qquad
\langle\phi_j|T|\phi_j\rangle = e^{-ik_j}\ \Rightarrow\
k_j=-\arg\big(\langle\phi_j|T|\phi_j\rangle\big).
$$
Then **snap** each $k_j$ to the nearest allowed lattice momentum $k_m=2\pi m/N$ and **assign** the energy $E_j$ to that momentum sector $m$.

In [6]:
def Oper(Ham, TransMat, alpha, k):
    """
    Args:
        Ham: Hamiltonian matrix
        TransMat: Translation operator matrix
        alpha: Coupling strength
        k: Momentum
    Returns:
        Oper: Combined operator matrix
    """
    return Ham - alpha * np.exp(-1j * k) * TransMat

def Dispersion(Ham, Transmat, alpha, k):
    """
    Args:
        Ham: Hamiltonian matrix 
        Transmat: Translation operator matrix
        alpha: Coupling strength
        k: Momentum
    Returns:
        energy: Energy eigenvalues
        k_out: Momentum values from phase of eigenvalues
        eigvec: Eigenvectors
    """
    oper = Oper(Ham, Transmat, alpha, k)
    eigval, eigvec = LA.eig(oper)
    energy = np.real(np.diag(np.dot(np.dot(eigvec.conj().T, Ham), eigvec)))
    phase = np.diag(np.dot(np.dot(eigvec.conj().T, Transmat), eigvec))
    k_out = -np.real(np.angle(phase))
    return energy, k_out, eigvec

## Result 1:
Calculate the dispersion relation $E(k)$ and show the eigenvalues in each momentum sector. And we find real ground state energy $E$ per site

In [7]:
Ns = 8
Params = [1, 1, 1]
Ham = ClusterIsingHam(Params, Ns)
TransMat = TranslationMat(Ns)
alpha = 1

E0 = None
psi0 = None
m0 = None

for m in range(Ns):
    k = 2*np.pi*m/Ns
    energy, k_out, eigvec = Dispersion(Ham, TransMat, alpha, k)
    m_out = (np.round(k_out * Ns / (2*np.pi)).astype(int)) % Ns
    mask = (m_out == m)
    energy_m = np.sort(energy[mask])   
    print(f"Momentum_Sector_ki={m}\n", np.round(energy_m, 10))
    vecs_m  = eigvec[:, mask]

    jloc = np.argmin(energy_m)
    Ej   = float(energy_m[jloc])
    psij = vecs_m[:, jloc]
    psij = psij / LA.norm(psij)

    # Find the ground state
    if (E0 is None) or (Ej < E0):
        E0 = Ej
        psi0 = psij
        m0 = m

print(f"Grounst state energy per site = {E0/Ns:.12f}")

Momentum_Sector_ki=0
 [-11.9745007  -11.89024988  -9.51415508  -9.25511722  -7.85248869
  -6.29085969  -5.05463167  -4.04429811  -3.37230998  -2.88661648
  -2.82842712  -2.4463952   -1.75455059  -1.46410162  -1.40352048
  -1.23238656  -0.42483153  -0.          -0.          -0.
   0.           0.88020197   0.89940195   2.11883867   2.82842712
   2.85399292   3.08806696   3.86534552   4.34331496   4.58045444
   5.46410162   6.55887379   7.22620043   8.38209486  11.13010482
  11.47002058]
Momentum_Sector_ki=1
 [-9.52059914 -9.14361028 -7.96292754 -6.285127   -4.56928847 -4.15230775
 -3.82710224 -2.76562845 -2.2658024  -1.7208036  -1.36281696 -1.25373809
 -1.09274357 -0.85649737 -0.71798957  0.12866451  0.70294386  1.24417659
  1.33391563  2.04239151  2.24284137  2.84133477  3.18876583  4.93304761
  4.95712202  5.34772734  5.36349783  6.68121886  7.16843394  9.32090075]
Momentum_Sector_ki=2
 [-9.75961101 -8.36870445 -7.13823818 -6.03408247 -5.13717262 -4.87955417
 -3.80934503 -3.78807751 -

## Result 2
Calculate the magnetization per site $\langle \sigma_i^z \rangle$ and $\langle \sigma_i^x \rangle$. Here we avoid matrix calculation but use bit operations.

In [8]:
def CalSigmaz(state, Ns):
    """
    Use bit manipulation to calculate sigma z per site
    Args:
        state: Wavefunction
        Ns: Number of sites
    Returns:
        mz: Sigma z per site
    """
    probs = np.abs(state)**2
    dim = 2 ** Ns
    mz = 0
    dim_array = np.arange(dim)
    for site in range(Ns):
        sigmaz = 2 * ReadBit(dim_array, site) - 1
        mz += np.dot(probs, sigmaz)
    mz /= Ns

    if mz < 1.0e-12:
        mz = 0
    return mz


def CalSigmax(state, Ns):
    """
    Use bit manipulation to calculate sigma x per site
    Args:
        state: Wavefunction
        Ns: Number of sites
    Returns:
        mx: Sigma x per site
    """
    mx = 0
    dim = 2 ** Ns
    dim_array = np.arange(dim)
    for site in range(Ns):
        flip_index = FlipBit(dim_array, site)
        mx += np.vdot(state, state[flip_index]).real
    mx /= Ns

    if mx < 1.0e-12:
        mx = 0
    return mx
mz = CalSigmaz(psi0, Ns)
mx = CalSigmax(psi0, Ns)
print(f"Ground state sigma x per site = {mx}")
print(f"Ground state sigma z per site = {mz}")


Ground state sigma x per site = 0.4569519539796918
Ground state sigma z per site = 0
